In [ ]:
# Import dependencies
import pandas as pd

In [ ]:
file_dir = 'C:/Users/ruchi/Desktop/Berkley Extension Learning Docs/Final Project'

In [ ]:
# Extract Kaggle Data
kaggle_metadata = pd.read_csv(f'{file_dir}/US_Accidents_Dec19.tar.gz', compression='gzip', error_bad_lines=False, low_memory=False)


# Data Extraction

In [ ]:
### Turn off SettingWithCopyWarning ##############
pd.options.mode.chained_assignment = None

In [ ]:
# Check total rows extraced
len(kaggle_metadata)

In [ ]:
# Extract only Columns relevant for analysis
df_subset = kaggle_metadata[['US_Accidents_Dec19.csv','Severity','Start_Time','End_Time',
                             'Start_Lat','Start_Lng','Distance(mi)', 'Street','Side','City',
                             'County','State','Zipcode','Timezone', 
                             'Temperature(F)','Humidity(%)','Pressure(in)',
                             'Visibility(mi)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)',
                             'Weather_Condition','Amenity','Crossing','Junction','Railway',
                             'Station','Stop','Traffic_Signal','Civil_Twilight'
                            ]]

# Transform

In [ ]:
# Check Null Values
df_subset.isnull().sum()

In [ ]:
# Fill NA with zero values for Precipitation column
df_subset["Precipitation(in)"].fillna(0, inplace = True) 

In [ ]:
# Drop rows with other NA values
df_subset.dropna(inplace=True)

In [ ]:
# Check the resulting dataset length
len(df_subset)

In [ ]:
# Sort the dataframe on Severity so that when removing duplicates the one with higher severity is retained
sorted_df = df_subset.sort_values('Severity',ascending=False)

In [ ]:
# Check how many duplicates exist in the dataset
len(sorted_df[['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng']].drop_duplicates())

In [ ]:
# Remove duplicates
sorted_df.drop_duplicates(subset=['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng'], inplace = True) 

In [ ]:
# Extract first 5 digits of zipcode where zip code is in postal format of ZIP-4
sorted_df['Zipcode'] = sorted_df['Zipcode'].str.replace(r"-.*","")

In [ ]:
# Check length of the remaining dataset after removing duplicate and Null value rows
len(sorted_df)

In [ ]:
# check datatypes for corrections
sorted_df.dtypes

In [ ]:
sorted_df['Start_Time'] = pd.to_datetime(sorted_df.Start_Time)
sorted_df['End_Time'] = pd.to_datetime(sorted_df.End_Time)

In [ ]:
# Change Datatype for Severity from float to integer
sorted_df['Severity'] = sorted_df['Severity'].astype(int)

In [ ]:
#########################   Create Highway Column   ####################################


searchfor = ['highway', 'Tollway', 'expy', 'fwy', 'hwy', 'Interstate', 
             'Tpke', 'Pkwy', 'Parkway', '-', 'US', 'Route', 
             'FM', 'Byp', 'Trwy', 'Beltway', 'Skyway', 'Skwy', ]
sorted_df.loc[sorted_df['Street'].str.contains('|'.join(searchfor), case=False), 'Highway'] = 'Y'

# Fill NA with zero values for Precipitation column
sorted_df["Highway"].fillna('N', inplace = True) 


In [ ]:
##sorted_df[['Start_Lat','Start_Lng']][sorted_df['Highway'] == 'N']

In [ ]:
# Create Coordinates column
sorted_df['Coordinates'] = sorted_df['Start_Lat'].map(str) + ':' + sorted_df['Start_Lng'].map(str)

In [ ]:
# Rename columns
sorted_df = sorted_df.rename(index=str,columns={'US_Accidents_Dec19.csv': 'Accident_ID'})

In [ ]:
# Create Dataframes for Loading into SQL Tables 

table1_df = sorted_df[['Accident_ID','Severity','Start_Time','End_Time',
                             'Start_Lat','Start_Lng','Coordinates', 'Distance(mi)', 'Side', 
                             'Temperature(F)','Humidity(%)','Pressure(in)',
                             'Visibility(mi)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)',
                             'Weather_Condition','Amenity','Crossing','Junction','Railway',
                             'Station','Stop','Traffic_Signal','Civil_Twilight'
                             ]]
table2_df = sorted_df[['Coordinates', 'Street','City','County','State','Zipcode',
                       'Timezone', 'Highway']]

In [ ]:
table2_df.drop_duplicates(subset=['Coordinates'], inplace=True)

In [ ]:
# Rename dataframe columns to match the table column names
table2_df.columns = map(str.lower, table2_df.columns)

In [ ]:
# Rename dataframe columns to match the table column names
table1_df = table1_df.rename(index=str,columns={'Distance(mi)':'distance', 
                             'Temperature(F)':'temperature',
                             'Humidity(%)':'humidity',
                             'Pressure(in)':'pressure',
                             'Visibility(mi)':'visibility',
                             'Wind_Speed(mph)':'wind_speed',
                             'Precipitation(in)':'precipitation',
                             })

table1_df.columns = map(str.lower, table1_df.columns)


In [ ]:
# Set Index
table1_df.set_index('accident_id', inplace=True)
table2_df.set_index('coordinates', inplace=True)

In [ ]:
# Since very large dataset Create CSVs to load data into Postgres SQL tables
table1_df.to_csv('table1.csv', sep='|')
table2_df.to_csv('table2.csv',sep='|')

# LOAD to SQL

In [ ]:
# Import SQL Load dependencies
#from sqlalchemy import create_engine
#import sqlalchemy as db
import psycopg2
from config import db_password

In [ ]:
conn = psycopg2.connect(
    host="accident-viz.c4cdhyeva5ut.us-east-1.rds.amazonaws.com", 
    port='5432', 
    dbname="Accident-ETL", 
    user="postgres", 
    password=db_password
)
cur = conn.cursor()

In [ ]:
with open('table2.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'accident_location', sep='|')
    conn.commit()

In [ ]:
with open('table1.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'accidents', sep='|')
    conn.commit()

#### Split Date and Time Column into Date, Time, Time in Seconds, and Day of week columns

In [ ]:
# new data frame with split value Start date time column
newstart = sorted_df["Start_Time"].str.split(" ",expand = True) 
  
# making separate Start Time column from new data frame 
sorted_df["Start_Time_of_Day"]= newstart[1] 

In [ ]:
# new data frame with split value End date time column
newend = sorted_df["End_Time"].str.split(" ",expand = True) 

# making separate Start Time column from new data frame 
sorted_df["End_Time_of_Day"]= newend[1] 

In [ ]:
# Convert Time to seconds for Start Time and End Time
sorted_df['Start_seconds'] = pd.to_timedelta(sorted_df['Start_Time_of_Day']).dt.seconds

# Convert Time to seconds for Start Time and End Time
sorted_df['End_seconds'] = pd.to_timedelta(sorted_df['End_Time_of_Day']).dt.seconds

In [ ]:
sorted_df['Start_Time'] = pd.to_datetime(sorted_df.Start_Time)
sorted_df['End_Time'] = pd.to_datetime(sorted_df.End_Time)

In [ ]:
# Get Day of the week for the accident
sorted_df['Day_of_Week'] = sorted_df['Start_Time'].dt.weekday

In [ ]:
sorted_df.head(5)

In [ ]:
sorted_df.dtypes

In [ ]:
sorted_df[sorted_df['US_Accidents_Dec19.csv'] == 'A-2782717']

In [ ]:
sorted_df.head(100).to_csv('Sample_data.csv')